# 0. 데이터 셋 호출 및 전처리

### Python 내장 csv 호출 모듈 사용 

### itemSetList 정의

In [25]:
from csv import reader

f = open(r'C:\Users\is041\geon_hyeon\Git\Apriori\dataset\kaggle.csv', 'r', encoding = 'utf-8')

# reader 가 한 라인씩 읽어오면서 뒤에 들여쓰기 까지 들고와서 리스트 맨 마지막 삭제
itemSetList = list(map(lambda x: x[:-1], reader(f)))[:4000] 

itemSetList[:5]

[['Lassi', 'Coffee Powder', 'Butter', 'Yougurt', 'Ghee', 'Cheese'],
 ['Ghee', 'Coffee Powder'],
 ['Lassi', 'Tea Powder', 'Butter', 'Cheese'],
 ['Cheese', 'Tea Powder', 'Panner', 'Coffee Powder', 'Butter', 'Bread'],
 ['Cheese', 'Yougurt', 'Coffee Powder', 'Sugar', 'Butter', 'Sweet']]

### itemSets, itemSet 정의

In [26]:
# List[List[str]] -> List[List[str]] but None 값 제거
itemSets = list(map(lambda x: list(filter(None, x)), itemSetList))

print(itemSets[0])

['Lassi', 'Coffee Powder', 'Butter', 'Yougurt', 'Ghee', 'Cheese']


In [27]:
from itertools import chain, combinations

# List[List[str]] -> Set[frozenset]
itemSet = set(map(lambda item: frozenset([item]), chain(*itemSetList)))

print(itemSet)

{frozenset({'Butter'}), frozenset({'Milk'}), frozenset({'Lassi'}), frozenset({'Coffee Powder'}), frozenset({'Yougurt'}), frozenset({'Ghee'}), frozenset({'Cheese'}), frozenset({'Sweet'}), frozenset({'Sugar'}), frozenset({'Tea Powder'}), frozenset({'Panner'}), frozenset({'Bread'})}


# 1. 데이터 나누기

In [11]:
# 이런식으로 쪼개거나 뭐...그냥 1000개씩 나눠서 돌아가는거만 일딴 확인

# def split_dict(input_dict: dict, num_parts: int) -> list:
#     list_len: int = len(input_dict)
#     return [(f'Node{i+2}', dict(list(input_dict.items())[i * list_len // num_parts:(i + 1) * list_len // num_parts]))
#         for i in range(num_parts)]
        
# N_data_blocks = split_dict(data, 3)

# N_data_blocks

In [18]:
# 이런식으로 4개로 나눠진 데이터 집합에서 각자 카운트를 진행하고 그 결과를 합쳐서 계산하자.

data_1 = itemSetList[:1000]
data_2 = itemSetList[1000:2000]
data_3 = itemSetList[2000:3000]
data_4 = itemSetList[3000:4000]

transaction_len_1 = len(data_1)
transaction_len_2 = len(data_2)
transaction_len_3 = len(data_3)
transaction_len_4 = len(data_4)

In [13]:
data_1

[['Lassi', 'Coffee Powder', 'Butter', 'Yougurt', 'Ghee', 'Cheese'],
 ['Ghee', 'Coffee Powder'],
 ['Lassi', 'Tea Powder', 'Butter', 'Cheese'],
 ['Cheese', 'Tea Powder', 'Panner', 'Coffee Powder', 'Butter', 'Bread'],
 ['Cheese', 'Yougurt', 'Coffee Powder', 'Sugar', 'Butter', 'Sweet'],
 ['Sugar', 'Tea Powder', 'Ghee', 'Sweet', 'Panner', 'Milk'],
 ['Sweet', 'Coffee Powder'],
 ['Butter', 'Ghee', 'Panner'],
 ['Sweet', 'Tea Powder', 'Butter', 'Yougurt', 'Sugar', 'Cheese'],
 ['Panner', 'Ghee'],
 ['Milk', 'Panner', 'Tea Powder', 'Sweet', 'Bread'],
 ['Ghee',
  'Coffee Powder',
  'Milk',
  'Yougurt',
  'Lassi',
  'Sugar',
  'Butter',
  'Panner'],
 ['Butter', 'Coffee Powder', 'Panner', 'Sweet', 'Ghee', 'Lassi'],
 ['Bread',
  'Lassi',
  'Coffee Powder',
  'Tea Powder',
  'Sweet',
  'Ghee',
  'Sugar',
  'Panner'],
 ['Milk', 'Sweet', 'Butter', 'Sugar', 'Lassi', 'Panner'],
 ['Bread', 'Coffee Powder', 'Tea Powder'],
 ['Butter', 'Ghee', 'Milk', 'Cheese'],
 ['Bread', 'Coffee Powder'],
 ['Cheese', 'Tea Po

# 2. 빈도 분석

In [147]:
from typing import List, Set, Dict, Tuple # 입출력 데이터의 형태를 표시해주는 정도의 역할
from itertools import combinations # 원소들의 조합을 계산하는 내장함수
from collections import defaultdict # dict에 key 값이 없으면 key값을 생성하고 초깃값을 0으로 세팅해준다.

def frequency_count(data: List[List[str]], max_value: int) -> Dict:
    frequency_values_dict = defaultdict(int)
    
    for transaction in data: # 1개의 원소로 이뤄진 값의 빈도 
        for key in transaction:
            frequency_values_dict[frozenset([key])] += 1 
        
    for i in range(2, max_value + 1):
        for transaction in data:
            combination = list(combinations(transaction, i)) # transaction에서 i개로 이뤄진 조합
            for key in combination:
                frequency_values_dict[frozenset(key)] += 1 
                
    return frequency_values_dict        

In [148]:
node_1 = frequency_count(data_1, 3)

node_1

defaultdict(int,
            {frozenset({'Lassi'}): 428,
             frozenset({'Coffee Powder'}): 439,
             frozenset({'Butter'}): 445,
             frozenset({'Yougurt'}): 458,
             frozenset({'Ghee'}): 448,
             frozenset({'Cheese'}): 414,
             frozenset({'Tea Powder'}): 429,
             frozenset({'Panner'}): 420,
             frozenset({'Bread'}): 424,
             frozenset({'Sugar'}): 425,
             frozenset({'Sweet'}): 424,
             frozenset({'Milk'}): 432,
             frozenset({'Coffee Powder', 'Lassi'}): 197,
             frozenset({'Butter', 'Lassi'}): 226,
             frozenset({'Lassi', 'Yougurt'}): 209,
             frozenset({'Ghee', 'Lassi'}): 196,
             frozenset({'Cheese', 'Lassi'}): 195,
             frozenset({'Butter', 'Coffee Powder'}): 195,
             frozenset({'Coffee Powder', 'Yougurt'}): 215,
             frozenset({'Coffee Powder', 'Ghee'}): 206,
             frozenset({'Cheese', 'Coffee Powder'}): 197,


In [149]:
node_2 = frequency_count(data_2, 3)

node_2

defaultdict(int,
            {frozenset({'Sweet'}): 425,
             frozenset({'Bread'}): 415,
             frozenset({'Milk'}): 439,
             frozenset({'Lassi'}): 436,
             frozenset({'Tea Powder'}): 396,
             frozenset({'Ghee'}): 442,
             frozenset({'Sugar'}): 426,
             frozenset({'Butter'}): 438,
             frozenset({'Coffee Powder'}): 446,
             frozenset({'Panner'}): 421,
             frozenset({'Cheese'}): 437,
             frozenset({'Yougurt'}): 428,
             frozenset({'Bread', 'Sweet'}): 193,
             frozenset({'Milk', 'Sweet'}): 190,
             frozenset({'Lassi', 'Sweet'}): 201,
             frozenset({'Bread', 'Milk'}): 184,
             frozenset({'Bread', 'Lassi'}): 192,
             frozenset({'Lassi', 'Milk'}): 192,
             frozenset({'Ghee', 'Tea Powder'}): 190,
             frozenset({'Sugar', 'Tea Powder'}): 179,
             frozenset({'Lassi', 'Tea Powder'}): 171,
             frozenset({'Sweet', 'T

In [150]:
node_3 = frequency_count(data_3, 3)

node_3

defaultdict(int,
            {frozenset({'Milk'}): 459,
             frozenset({'Panner'}): 444,
             frozenset({'Lassi'}): 428,
             frozenset({'Ghee'}): 488,
             frozenset({'Cheese'}): 458,
             frozenset({'Coffee Powder'}): 430,
             frozenset({'Tea Powder'}): 423,
             frozenset({'Sugar'}): 421,
             frozenset({'Sweet'}): 439,
             frozenset({'Yougurt'}): 427,
             frozenset({'Bread'}): 462,
             frozenset({'Butter'}): 431,
             frozenset({'Milk', 'Panner'}): 210,
             frozenset({'Lassi', 'Milk'}): 206,
             frozenset({'Ghee', 'Milk'}): 232,
             frozenset({'Cheese', 'Milk'}): 216,
             frozenset({'Coffee Powder', 'Milk'}): 198,
             frozenset({'Lassi', 'Panner'}): 189,
             frozenset({'Ghee', 'Panner'}): 228,
             frozenset({'Cheese', 'Panner'}): 217,
             frozenset({'Coffee Powder', 'Panner'}): 196,
             frozenset({'Ghee'

In [151]:
node_4 = frequency_count(data_4, 3)

node_4

defaultdict(int,
            {frozenset({'Ghee'}): 427,
             frozenset({'Bread'}): 438,
             frozenset({'Tea Powder'}): 439,
             frozenset({'Sweet'}): 446,
             frozenset({'Panner'}): 443,
             frozenset({'Coffee Powder'}): 463,
             frozenset({'Yougurt'}): 431,
             frozenset({'Butter'}): 449,
             frozenset({'Sugar'}): 461,
             frozenset({'Cheese'}): 425,
             frozenset({'Milk'}): 442,
             frozenset({'Lassi'}): 433,
             frozenset({'Bread', 'Ghee'}): 190,
             frozenset({'Ghee', 'Tea Powder'}): 193,
             frozenset({'Ghee', 'Sweet'}): 189,
             frozenset({'Bread', 'Tea Powder'}): 191,
             frozenset({'Bread', 'Sweet'}): 200,
             frozenset({'Sweet', 'Tea Powder'}): 194,
             frozenset({'Coffee Powder', 'Panner'}): 220,
             frozenset({'Bread', 'Panner'}): 206,
             frozenset({'Panner', 'Sweet'}): 206,
             frozenset(

# 3. merge

- 이 부분에 대한 내용을 좀 더 수정하고 싶다. 단순한 합을 구하여 결과를 도출하는 것은 매우 비효율적인 것 같다.
- 굳이 이 과정이 필요한가? 처음부터 합하지 않은 상태로 기록하고, 연관규칙분석의 결과를 도출할때 상황에 맞게 해석하는 것이 훨씬 더 효율적이지 않을까 ?

## 3.1 단순한 합

In [152]:
transaction_all_len = transaction_len_1 + transaction_len_2 + transaction_len_3 + transaction_len_4

transaction_all_len

4000

In [153]:
from collections import Counter

node_sum = dict(Counter(node_1) + Counter(node_2) + Counter(node_3) + Counter(node_4))

node_sum

{frozenset({'Lassi'}): 1725,
 frozenset({'Coffee Powder'}): 1778,
 frozenset({'Butter'}): 1763,
 frozenset({'Yougurt'}): 1744,
 frozenset({'Ghee'}): 1805,
 frozenset({'Cheese'}): 1734,
 frozenset({'Tea Powder'}): 1687,
 frozenset({'Panner'}): 1728,
 frozenset({'Bread'}): 1739,
 frozenset({'Sugar'}): 1733,
 frozenset({'Sweet'}): 1734,
 frozenset({'Milk'}): 1772,
 frozenset({'Coffee Powder', 'Lassi'}): 801,
 frozenset({'Butter', 'Lassi'}): 798,
 frozenset({'Lassi', 'Yougurt'}): 790,
 frozenset({'Ghee', 'Lassi'}): 815,
 frozenset({'Cheese', 'Lassi'}): 794,
 frozenset({'Butter', 'Coffee Powder'}): 821,
 frozenset({'Coffee Powder', 'Yougurt'}): 803,
 frozenset({'Coffee Powder', 'Ghee'}): 869,
 frozenset({'Cheese', 'Coffee Powder'}): 791,
 frozenset({'Butter', 'Yougurt'}): 827,
 frozenset({'Butter', 'Ghee'}): 848,
 frozenset({'Butter', 'Cheese'}): 779,
 frozenset({'Ghee', 'Yougurt'}): 826,
 frozenset({'Cheese', 'Yougurt'}): 792,
 frozenset({'Cheese', 'Ghee'}): 790,
 frozenset({'Lassi', 'Tea 

# 4. 연관규칙분석 평가척도 계산

## 4.1 Support

### 4.1.1 소프트웨어 선호방식
- Support(A -> B) = P(A, B)

In [52]:
print('Support - (Ghee, Bread): ', node_sum['Ghee','Bread']/transaction_all_len)
print('Support - (Coffee Powder, Lassi): ',node_sum['Coffee Powder','Lassi']/transaction_all_len)
print('Support - (Ghee, Lassi): ',node_sum['Ghee','Lassi']/transaction_all_len)
print('Support - (Coffee Powder, Yougurt): ',node_sum['Coffee Powder','Yougurt']/transaction_all_len)
print('Support - (Ghee, Panner): ',node_sum['Ghee','Panner']/transaction_all_len)

Support - (Ghee, Bread):  0.1025
Support (Coffee Powder, Lassi):  0.10325
Support (Ghee, Lassi):  0.1025
Support (Coffee Powder, Yougurt):  0.1015
Support (Ghee, Panner):  0.104


### 4.1.2 전통적인 방식
Support(A -> B) = P(A)

In [53]:
print('Support - (Ghee, Bread): ', node_sum['Ghee']/transaction_all_len)
print('Support - (Coffee Powder, Lassi): ',node_sum['Coffee Powder']/transaction_all_len)
print('Support - (Ghee, Lassi): ',node_sum['Ghee']/transaction_all_len)
print('Support - (Coffee Powder, Yougurt): ',node_sum['Coffee Powder']/transaction_all_len)
print('Support - (Ghee, Panner): ',node_sum['Ghee']/transaction_all_len)

Support - (Ghee, Bread):  0.45125
Support - (Coffee Powder, Lassi):  0.4445
Support - (Ghee, Lassi):  0.45125
Support - (Coffee Powder, Yougurt):  0.4445
Support - (Ghee, Panner):  0.45125


## 4.2 Confidence
- PyPi 에서 나와있는 알고리즘 동작과정에서는 전통적인 A priori 알고리즘 계산 방식과는 조금 다르게, Confidence 값 중 최대값을 기준으로 계산한다. ex) A -> B : 0.7, B -> A: 0.56 이면 Confidence(A, B) = 0.7이다.
- Confidence(A -> B) = Support(A & B) / Support(A)

In [54]:
print('Confidence - (Ghee, Bread): ', node_sum['Ghee','Bread']/node_sum['Ghee'])
print('Confidence - (Coffee Powder, Lassi): ',node_sum['Coffee Powder','Lassi']/node_sum['Coffee Powder'])
print('Confidence - (Ghee, Lassi): ',node_sum['Ghee','Lassi']/node_sum['Ghee'])
print('Confidence - (Coffee Powder, Yougurt): ',node_sum['Coffee Powder','Yougurt']/node_sum['Coffee Powder'])
print('Confidence - (Ghee, Panner): ',node_sum['Ghee','Panner']/node_sum['Ghee'])

Confidence - (Ghee, Bread):  0.22714681440443213
Confidence - (Coffee Powder, Lassi):  0.23228346456692914
Confidence - (Ghee, Lassi):  0.22714681440443213
Confidence - (Coffee Powder, Yougurt):  0.2283464566929134
Confidence - (Ghee, Panner):  0.23047091412742382


## 4.3 PyPi에서 들고온 방식대로 계산하기
- 아아 내가 착각했다. 이거 Support(A & B)이면, Support(A, B) + Support(B, A)인 결과를 가지고 평가척도 결과에 반영해야된다.

In [162]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))            
            
def associationRule_just_one(item):
    rules = []
    subsets = powerset(item)
    
    for s in subsets:
        confidence = float(
            node_sum[item] / node_sum[frozenset(s)])
        rules.append([set(s), set(item.difference(s)), confidence])
        
    return rules

In [164]:
print(associationRule_just_one(frozenset(('Ghee','Bread'))))
print(associationRule_just_one(frozenset(('Coffee Powder','Lassi'))))
print(associationRule_just_one(frozenset(('Ghee','Lassi'))))
print(associationRule_just_one(frozenset(('Coffee Powder','Yougurt'))))
print(associationRule_just_one(frozenset(('Ghee','Panner'))))

frozenset({'Ghee', 'Bread'})
[[{'Ghee'}, {'Bread'}, 0.44986149584487534], [{'Bread'}, {'Ghee'}, 0.4669350201265095]]
frozenset({'Lassi', 'Coffee Powder'})
[[{'Lassi'}, {'Coffee Powder'}, 0.4643478260869565], [{'Coffee Powder'}, {'Lassi'}, 0.45050618672665915]]
frozenset({'Lassi', 'Ghee'})
[[{'Lassi'}, {'Ghee'}, 0.47246376811594204], [{'Ghee'}, {'Lassi'}, 0.4515235457063712]]
frozenset({'Coffee Powder', 'Yougurt'})
[[{'Coffee Powder'}, {'Yougurt'}, 0.4516310461192351], [{'Yougurt'}, {'Coffee Powder'}, 0.46043577981651373]]
frozenset({'Panner', 'Ghee'})
[[{'Panner'}, {'Ghee'}, 0.47280092592592593], [{'Ghee'}, {'Panner'}, 0.45263157894736844]]


# A priori

## 1. 필요한 함수들 정의

In [32]:
from collections import defaultdict
from typing import List, Set, Dict, Tuple, FrozenSet

def getAboveMinSup(itemSet: Set[FrozenSet], itemSetList: List[List[str]], minSup: float, globalItemSetWithSup: Dict[str, int]) -> Set[FrozenSet]:
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1 # item 이 존재하면, 그 값을 globalSup, localSup에 각각 1씩 추가해라

    for item, supCount in localItemSetWithSup.items(): # key: item, value: supCount이다. items()는 key, value로 이뤄진 사전값 
        support = float(supCount / len(itemSetList)) # item 원소의 개수/ 전체 원소의 개수 
        if(support >= minSup):
            freqItemSet.add(item) # minSup의 값보다 큰 값만 freqitemSet에 저장

    return freqItemSet

In [33]:
def getUnion(itemSet: Set[FrozenSet], length): ## 합집합 구하기
    # length 즉, k의 길이 만큼에 해당하는 집합에 대한 집합이다.
    # ex) k = 2 라면, {'Bread', 'Sugar'} 이런식으로 중복이 없는, 2개의 원소만 가지는 집합에 대한 집합을 반환
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]) 

In [34]:
def pruning(candidateSet, prevFreqSet, length): ## 가지치기
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet

In [35]:
def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules

In [36]:
def getItemSetFromList(itemSetList): # 음 내 마음에 안든다 좀 바꿔봐야겠다.
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

In [37]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

## 2. A priori 알고리즘

In [38]:
def apriori(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2]) # confidence가 높은 순서대로 정렬

    return globalFreqItemSet, rules

## 3. 결과

In [39]:
freqItemSet, rules = apriori(itemSetList, minSup=0.2, minConf=0.4)

print(rules)

[[{'Ghee'}, {'Bread'}, 0.44986149584487534], [{'Coffee Powder'}, {'Lassi'}, 0.45050618672665915], [{'Ghee'}, {'Lassi'}, 0.4515235457063712], [{'Coffee Powder'}, {'Yougurt'}, 0.4516310461192351], [{'Ghee'}, {'Panner'}, 0.45263157894736844], [{'Coffee Powder'}, {'Bread'}, 0.453318335208099], [{'Milk'}, {'Yougurt'}, 0.45428893905191875], [{'Ghee'}, {'Sweet'}, 0.4548476454293629], [{'Milk'}, {'Sugar'}, 0.4548532731376975], [{'Milk'}, {'Sweet'}, 0.4548532731376975], [{'Milk'}, {'Bread'}, 0.45711060948081267], [{'Butter'}, {'Panner'}, 0.4571752694271129], [{'Ghee'}, {'Yougurt'}, 0.45761772853185595], [{'Milk'}, {'Butter'}, 0.4582392776523702], [{'Ghee'}, {'Sugar'}, 0.4587257617728532], [{'Yougurt'}, {'Sugar'}, 0.4598623853211009], [{'Yougurt'}, {'Coffee Powder'}, 0.46043577981651373], [{'Butter'}, {'Sugar'}, 0.46057855927396485], [{'Butter'}, {'Milk'}, 0.46057855927396485], [{'Yougurt'}, {'Milk'}, 0.46158256880733944], [{'Coffee Powder'}, {'Butter'}, 0.46175478065241843], [{'Sugar'}, {'Yougu

#### 상위 5개

- [{'Ghee'}, {'Bread'}, 0.44986149584487534], 
- [{'Coffee Powder'}, {'Lassi'}, 0.45050618672665915], 
- [{'Ghee'}, {'Lassi'}, 0.4515235457063712], 
- [{'Coffee Powder'}, {'Yougurt'}, 0.4516310461192351], 
- [{'Ghee'}, {'Panner'}, 0.45263157894736844]

# 음...그냥 무식하게 print 찍어보기

In [113]:
def associationRule_test(freqItemSet, itemSetWithSup, minConf): 
    '''
    
    freqltemSet <- 
    itemSetWithSup <- minSup 값 넘은 규칙의 집합
    
    
    '''
    print("#"*30, "associationRule", "#"*30)
    print("freqItemSet: ", freqItemSet)
    print('-'*100)
    
    print("itemSetWithSup: ", itemSetWithSup)
    print('-'*100)
    
    print("minConf: ", minConf)
    print('-'*100)
    
    rules = []
    for k, itemSet in freqItemSet.items(): # freqltemSet.items() <- Set 구조에서 하나씩 뽑아주는 것
        print("k", k,"itemSet", itemSet)
        for item in itemSet:
            print(item)
            subsets = powerset(item) # powerset() <- 멱집합(주어진 집합의 모든 부분집합의 집합)
            print(subsets)
            for s in subsets:
                print("s", s)
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules

In [114]:
def apriori_test(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    print('C1ItemSet',C1ItemSet)
    print('-'*100)
    
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    print('globalFreqItemSet',globalFreqItemSet)
    print('-'*100)
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)
    print('globalItemSetWithSup',globalItemSetWithSup)
    print('-'*100)
    
    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    print('currentLSet',currentLSet)
    print('-'*100)
    
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        print("#"*30, k, "#"*30)
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        print('currentLSet',currentLSet)
        print('-'*100)
        
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        print('candidateSet - getUnion',candidateSet)
        print('-'*100)
        
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        print('candidateSet - pruning',candidateSet)
        print('-'*100)
        
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        print('currentLSet - getAbiveMinSup',candidateSet)
        print('-'*100)
        
        k += 1

    rules = associationRule_test(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2]) # confidence가 높은 순서대로 정렬

    return globalFreqItemSet, rules

In [115]:
apriori_test(itemSetList, minSup=0.2, minConf=0.4)

C1ItemSet {frozenset({'Butter'}), frozenset({'Milk'}), frozenset({'Lassi'}), frozenset({'Coffee Powder'}), frozenset({'Yougurt'}), frozenset({'Ghee'}), frozenset({'Cheese'}), frozenset({'Sweet'}), frozenset({'Sugar'}), frozenset({'Tea Powder'}), frozenset({'Panner'}), frozenset({'Bread'})}
----------------------------------------------------------------------------------------------------
globalFreqItemSet {}
----------------------------------------------------------------------------------------------------
globalItemSetWithSup defaultdict(<class 'int'>, {})
----------------------------------------------------------------------------------------------------
currentLSet {frozenset({'Butter'}), frozenset({'Milk'}), frozenset({'Lassi'}), frozenset({'Coffee Powder'}), frozenset({'Yougurt'}), frozenset({'Ghee'}), frozenset({'Cheese'}), frozenset({'Sweet'}), frozenset({'Sugar'}), frozenset({'Panner'}), frozenset({'Tea Powder'}), frozenset({'Bread'})}
----------------------------------------

({1: {frozenset({'Butter'}),
   frozenset({'Milk'}),
   frozenset({'Lassi'}),
   frozenset({'Coffee Powder'}),
   frozenset({'Yougurt'}),
   frozenset({'Ghee'}),
   frozenset({'Cheese'}),
   frozenset({'Sweet'}),
   frozenset({'Sugar'}),
   frozenset({'Panner'}),
   frozenset({'Tea Powder'}),
   frozenset({'Bread'})},
  2: {frozenset({'Butter', 'Sugar'}),
   frozenset({'Milk', 'Yougurt'}),
   frozenset({'Bread', 'Panner'}),
   frozenset({'Butter', 'Panner'}),
   frozenset({'Milk', 'Sugar'}),
   frozenset({'Bread', 'Milk'}),
   frozenset({'Butter', 'Milk'}),
   frozenset({'Ghee', 'Yougurt'}),
   frozenset({'Coffee Powder', 'Ghee'}),
   frozenset({'Coffee Powder', 'Yougurt'}),
   frozenset({'Butter', 'Sweet'}),
   frozenset({'Milk', 'Sweet'}),
   frozenset({'Coffee Powder', 'Lassi'}),
   frozenset({'Butter', 'Coffee Powder'}),
   frozenset({'Butter', 'Yougurt'}),
   frozenset({'Butter', 'Ghee'}),
   frozenset({'Bread', 'Ghee'}),
   frozenset({'Ghee', 'Sweet'}),
   frozenset({'Sugar', 'Yo

# apriori_python 직접 설치 후 실행

!pip install apriori_python

In [40]:
# from apriori_python import apriori
# itemSetList = [['eggs', 'bacon', 'soup'],
#                 ['eggs', 'bacon', 'apple'],
#                 ['soup', 'bacon', 'banana']]
# freqItemSet, rules = apriori(itemSetList, minSup=0.5, minConf=0.5)
# print(rules)  
# # [[{'beer'}, {'rice'}, 0.6666666666666666], [{'rice'}, {'beer'}, 1.0]]
# # rules[0] --> rules[1], confidence = rules[2]

In [42]:
from apriori_python import apriori
from csv import reader

f = open(r'C:\Users\is041\geon_hyeon\Git\Apriori\dataset\kaggle.csv', 'r', encoding = 'utf-8')

# reader 가 한 라인씩 읽어오면서 뒤에 들여쓰기 까지 들고와서 리스트 맨 마지막 삭제
itemSetList = list(map(lambda x: x[:-1], list(reader(f))))[:4000] 

freqItemSet, rules = apriori(itemSetList, minSup=0.2, minConf=0.4)

print(rules)

[[{'Ghee'}, {'Bread'}, 0.44986149584487534], [{'Coffee Powder'}, {'Lassi'}, 0.45050618672665915], [{'Ghee'}, {'Lassi'}, 0.4515235457063712], [{'Coffee Powder'}, {'Yougurt'}, 0.4516310461192351], [{'Ghee'}, {'Panner'}, 0.45263157894736844], [{'Coffee Powder'}, {'Bread'}, 0.453318335208099], [{'Milk'}, {'Yougurt'}, 0.45428893905191875], [{'Ghee'}, {'Sweet'}, 0.4548476454293629], [{'Milk'}, {'Sugar'}, 0.4548532731376975], [{'Milk'}, {'Sweet'}, 0.4548532731376975], [{'Milk'}, {'Bread'}, 0.45711060948081267], [{'Butter'}, {'Panner'}, 0.4571752694271129], [{'Ghee'}, {'Yougurt'}, 0.45761772853185595], [{'Milk'}, {'Butter'}, 0.4582392776523702], [{'Ghee'}, {'Sugar'}, 0.4587257617728532], [{'Yougurt'}, {'Sugar'}, 0.4598623853211009], [{'Yougurt'}, {'Coffee Powder'}, 0.46043577981651373], [{'Butter'}, {'Sugar'}, 0.46057855927396485], [{'Butter'}, {'Milk'}, 0.46057855927396485], [{'Yougurt'}, {'Milk'}, 0.46158256880733944], [{'Coffee Powder'}, {'Butter'}, 0.46175478065241843], [{'Sugar'}, {'Yougu

#### 상위 5개

- [{'Ghee'}, {'Bread'}, 0.44986149584487534], 
- [{'Coffee Powder'}, {'Lassi'}, 0.45050618672665915], 
- [{'Ghee'}, {'Lassi'}, 0.4515235457063712], 
- [{'Coffee Powder'}, {'Yougurt'}, 0.4516310461192351], 
- [{'Ghee'}, {'Panner'}, 0.45263157894736844]

In [56]:
print(freqItemSet)

{1: {frozenset({'Butter'}), frozenset({'Milk'}), frozenset({'Lassi'}), frozenset({'Coffee Powder'}), frozenset({'Yougurt'}), frozenset({'Ghee'}), frozenset({'Cheese'}), frozenset({'Sweet'}), frozenset({'Sugar'}), frozenset({'Panner'}), frozenset({'Tea Powder'}), frozenset({'Bread'})}, 2: {frozenset({'Sugar', 'Butter'}), frozenset({'Milk', 'Yougurt'}), frozenset({'Panner', 'Bread'}), frozenset({'Panner', 'Butter'}), frozenset({'Sugar', 'Milk'}), frozenset({'Milk', 'Bread'}), frozenset({'Milk', 'Butter'}), frozenset({'Ghee', 'Yougurt'}), frozenset({'Ghee', 'Coffee Powder'}), frozenset({'Coffee Powder', 'Yougurt'}), frozenset({'Sweet', 'Butter'}), frozenset({'Milk', 'Sweet'}), frozenset({'Lassi', 'Coffee Powder'}), frozenset({'Coffee Powder', 'Butter'}), frozenset({'Butter', 'Yougurt'}), frozenset({'Ghee', 'Butter'}), frozenset({'Ghee', 'Bread'}), frozenset({'Ghee', 'Sweet'}), frozenset({'Sugar', 'Yougurt'}), frozenset({'Coffee Powder', 'Bread'}), frozenset({'Cheese', 'Panner'}), frozense